In [ ]:
import json
import os

def load_coco_data(folder_name):
    # Define the path to the JSON file
    json_file = os.path.join(folder_name, '_annotations.coco.json')
    
    # Check if the JSON file exists
    if not os.path.exists(json_file):
        print(f"JSON file not found in folder {folder_name}")
        return

    # Load COCO data from the JSON file
    with open(json_file, 'r') as file:
        coco_data = json.load(file)

    # Access various parts of the COCO data
    info = coco_data['info']
    licenses = coco_data['licenses']
    categories = coco_data['categories']
    images = coco_data['images']
    annotations = coco_data['annotations']

    # Example: Print the first image's filename
    if images:
        first_image_filename = images[0]['file_name']
        print(f"First image filename in {folder_name}: {first_image_filename}")

    # Example: Iterate through annotations and print category IDs
    for annotation in annotations:
        category_id = annotation['category_id']
        print(f"Annotation ID {annotation['id']} in {folder_name} belongs to category {category_id}")

    # Example: Print the categories
    for category in categories:
        category_name = category['name']
        print(f"Category ID {category['id']} in {folder_name} is named {category_name}")

# Define the folder names
folders = ['train', 'test', 'valid']

# Load COCO data for each folder
for folder_name in folders:
    load_coco_data(folder_name)


In [7]:
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
from PIL import Image, ImageDraw
import os

# Function to save mask images
def save_mask_image(image, annotations, output_dir, folder_name, file_name):
    # Create a blank mask image
    mask_image = Image.new('L', image.size, 0)

    # Draw masks on the mask image
    for annotation in annotations:
        mask = coco.annToMask(annotation)
        mask_image = Image.fromarray(mask * 255)
        mask_image = mask_image.convert('L')
        mask_image = mask_image.point(lambda p: p > 128 and 255)
        mask_image = Image.composite(mask_image, mask_image, Image.new('L', mask_image.size))

    # Save the mask image with folder name and picture name
    mask_filename = f'mask_{folder_name}_{file_name}'
    mask_image.save(os.path.join(output_dir, mask_filename))

# Initialize the COCO dataset for train, test, and valid
folder_names = ['train', 'test', 'valid']

for folder_name in folder_names:
    # Load the annotation file from the current folder
    annotation_file = os.path.join(folder_name, f'_annotations.coco.json')
    coco = COCO(annotation_file)

    # Create a directory for mask data
    output_dir = f'mask_data_{folder_name}'
    os.makedirs(output_dir, exist_ok=True)

    # Iterate through images in the current folder
    image_ids = coco.getImgIds()
    for image_id in image_ids:
        # Load image information
        image_info = coco.loadImgs(image_id)[0]
        file_name = image_info['file_name']
        image = Image.open(os.path.join(folder_name, file_name))  # Replace with the path to your image folder

        # Get annotations for the current image
        annotation_ids = coco.getAnnIds(imgIds=image_id)
        annotations = coco.loadAnns(annotation_ids)

        # Save the mask image with folder name and picture name
        save_mask_image(image, annotations, output_dir, folder_name, file_name)

    print(f'Mask images created and saved in the "mask_data_{folder_name}" directory.')


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Mask images created and saved in the "mask_data_train" directory.
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Mask images created and saved in the "mask_data_test" directory.
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Mask images created and saved in the "mask_data_valid" directory.
